In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

# ws = Workspace.get(name="quick-starts-ws-145499", 
#             subscription_id="9a7511b8-150f-4a58-8528-3e7d50216c31",
#             resource_group="aml-quickstarts-145499")
exp = Experiment(workspace=ws, name="bank_marketing")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-146476
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-146476


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# compute_target = ws.compute_targets['com1']
try:
    compute_target = ComputeTarget(workspace=ws, name="computeCluster")

except ComputeTargetException:
    compute_target = ComputeTarget.create(ws, 'computeCluster', AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4))
# print(ws.compute_targets)

compute_target.wait_for_completion(show_output=True)

Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import uniform, choice
import os
from azureml.core import ScriptRunConfig


# Specify parameter sampler
ps = RandomParameterSampling({
        "--C": uniform(0.1, 1),
        "--max_iter": choice(50, 75, 100, 150)
})
    ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# env = Environment()
# Create a SKLearn estimator for use with train.py

est = SKLearn(source_directory='./', 
    compute_target=compute_target, 
    entry_script='train.py',
    vm_size = "Standard_D2_V2")
# est = ScriptRunConfig(source_directory='.',
#                       script='train.py',
#                       arguments=['--C', 1.0, '--max_iter', 100],
#                       compute_target=compute_target
#                       )

# # Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
            estimator=est,
            policy=policy,
            hyperparameter_sampling=ps,
            primary_metric_name='Accuracy',
            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
            max_total_runs=120,
            max_concurrent_runs=4
)
    ### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
hdr_exp = exp.submit(hyperdrive_config)

In [6]:
RunDetails(hdr_exp).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
import joblib
# Get your best run and save the model from that run.
best_run_hdr = hdr_exp.get_best_run_by_primary_metric()
best_run_hdr.get_details()
print(hdr_exp.get_children_sorted_by_primary_metric(top=1))
### YOUR CODE HERE ###
# joblib.dump(best_run_hdr, './outputs/hdr_best_run_model.joblib')

[{'run_id': 'HD_66b240e1-0091-4a24-b32a-688ada419e0d_32', 'hyperparameters': '{"--C": 0.7736555084975076, "--max_iter": 50}', 'best_primary_metric': 0.9103692463328276, 'status': 'Completed'}]


In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
### YOUR CODE HERE ###

In [14]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)

In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=pd.concat([x,y], axis = 1),
    label_column_name='y',
    n_cross_validations=3)

In [19]:
# Submit your automl run
automl_exp = exp.submit(automl_config)
RunDetails(automl_exp).show()
### YOUR CODE HERE ###

Experiment,Id,Type,Status,Details Page,Docs Page
bank_marketing,AutoML_a14cfc54-e71c-4b1f-876d-bec8c3850e68,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [23]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run_automl = automl_exp.get_output()

best_run_automl[0].get_details()
# automl_exp.get_children_sorted_by_primary_metric(top=1)
# ### YOUR CODE HERE ###
joblib.dump(best_run_automl[1], './outputs/automl_best_run_model.joblib')

['./outputs/automl_best_run_model.joblib']